In [19]:
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
import pandas as pd
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct
from sentence_transformers import SentenceTransformer
from bs4 import BeautifulSoup
import uuid
import os
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

# ✅ Assign the API key
openai_api_key = os.getenv("OPENAI_API_KEY")

# ✅ Use it here
openai_client = OpenAI(
    api_key=openai_api_key
)

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PayloadSchemaType

# 1. Connect to Qdrant
client = QdrantClient(host="vps.maestri.com.co", port=6333, https=False)

# 2. Collection name and vector size
collection_name = "user_history"
embedding_size = 1536  # If you're using OpenAI's text-embedding-3-small

# 3. Check if the collection exists and delete it (optional)
if client.collection_exists(collection_name):
    print(f"🗑️ Deleting existing collection '{collection_name}'...")
    client.delete_collection(collection_name)

# 4. Create collection
print(f"🛠️ Creating collection '{collection_name}'...")
client.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=embedding_size, distance=Distance.COSINE),
    on_disk_payload=True
)
print(f"✅ Collection '{collection_name}' created.")

# 5. Define and create payload indexes (metadata fields)
fields = {
    "question": PayloadSchemaType.TEXT,
    "answer": PayloadSchemaType.TEXT,
    "subscriber_id": PayloadSchemaType.KEYWORD,
    "product_ids": PayloadSchemaType.KEYWORD,  # list of product slugs or IDs
    "timestamp": PayloadSchemaType.TEXT
}

print("🔧 Creating payload indexes...")
for field_name, field_type in fields.items():
    client.create_payload_index(
        collection_name=collection_name,
        field_name=field_name,
        field_schema=field_type
    )
print("✅ All payload indexes created successfully.")


🛠️ Creating collection 'user_history'...
✅ Collection 'user_history' created.
🔧 Creating payload indexes...
✅ All payload indexes created successfully.


In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import PointStruct, VectorParams, Distance
from bs4 import BeautifulSoup
import pandas as pd
import uuid
from openai import OpenAI # ✅ add openai import


# Utility functions
def clean(text):
    if pd.isna(text): return ""
    return str(text).strip()

def strip_html(text):
    return BeautifulSoup(text, "html.parser").get_text(separator=" ", strip=True)


def get_openai_embedding(text: str) -> list:
    response = openai_client.embeddings.create(
        model="text-embedding-3-small",
        input=text
    )
    return response.data[0].embedding

# Load CSV
df = pd.read_csv("/Users/diegohernandez/Downloads/Maestri Milano - Products.csv")
df.columns = [col.strip().lower().replace(" ", "_") for col in df.columns]

# Connect to Qdrant
client = QdrantClient(host="vps.maestri.com.co", port=6333, https=False)
collection_name = "maestri_products"
embedding_size = 1536  # ✅ OpenAI embedding size is 1536

client.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=embedding_size, distance=Distance.COSINE),
)

points = []

for _, row in df.iterrows():
    if pd.isna(row.get("published_on")):
        continue

    product_name = clean(row.get("nombre"))
    bodega = clean(row.get("bodega"))
    region = clean(row.get("región"))
    tipo = clean(row.get("tipo"))
    maridaje1 = clean(row.get("maridaje_1"))
    maridaje2 = clean(row.get("maridaje_2"))
    maridaje = " y ".join([m for m in [maridaje1, maridaje2] if m])
    notas = clean(row.get("notas_de_cata"))
    descripcion = strip_html(clean(row.get("descripción")))
    precio = clean(row.get("precio"))
    category = clean(row.get("pasillo"))
    alternate_names = ""  

    short_text = f"""\
                Producto: {product_name}. Tipo: {tipo}. Bodega: {bodega}. Región: {region}.
                Maridaje: {maridaje}. Notas: {notas}. Descripción: {descripcion}.
                También conocido como: {alternate_names}. Precio: {precio}.
                """

    if not short_text.strip():
        continue

    # ✅ Now using OpenAI to generate the embedding
    vector = get_openai_embedding(short_text)

    payload = {
        "product_name": product_name,
        "bodega": bodega,
        "region": region,
        "tipo": tipo,
        "precio": precio,
        "notas": notas,
        "descripcion": descripcion,
        "maridaje": maridaje,
        "category": category,
        "alternate_names": alternate_names
    }

    points.append(PointStruct(id=str(uuid.uuid4()), vector=vector, payload=payload))

# Insert into Qdrant
client.upsert(collection_name=collection_name, points=points)
print(f"✅ Inserted {len(points)} products into Qdrant collection: {collection_name}")


/var/folders/xq/gclpc2_d3ql92h6tddvfwkjr0000gn/T/ipykernel_30796/3315711371.py:34: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


ResponseHandlingException: [Errno 8] nodename nor servname provided, or not known

In [ ]:
df.columns

Index(['nombre', 'slug', 'collection_id', 'locale_id', 'item_id', 'created_on',
       'updated_on', 'published_on', 'precio', 'imagen_del_producto',
       'categoria', 'descripción', 'maridaje_1', 'maridaje_2', 'notas_de_cata',
       'temperatura_de_servicio', 'pasillo', 'tipo', 'región',
       'cepa_principal', 'ocasión', 'bodega', 'denominación', 'peso_/_volumen',
       'gr/ml', 'item_id', 'precio_descuento', 'categories', 'precios',
       'descuento', 'descuento_2x1', 'descuento_3x2', 'ciudad',
       'productoreserva', 'descuento%off'],
      dtype='object')

In [ ]:
# Step 1 - Inspect products with "prosecco" in product_name
print("\n🔍 Checking indexed Prosecco products by name:\n")

results, _ = client.scroll(
    collection_name=collection_name,
    limit=100,
    with_payload=True
)

prosecco_items = [
    r for r in results if "prosecco" in str(r.payload.get("product_name", "")).lower()
]

if not prosecco_items:
    print("❌ No Prosecco products found in product_name.")
else:
    for i, item in enumerate(prosecco_items, 1):
        print(f"\n#{i}")
        for k, v in item.payload.items():
            print(f"{k}: {v}")



🔍 Checking indexed Prosecco products by name:


#1
product_name: Prosecco Sior Piero
bodega: V8+
region: Veneto
tipo: Espumoso
precio: 115000.0
notas: De un color pajizo brillante, burbujas finas y persistentes, aromas delicados que recuerdan notas Florales y frutales que recuerdan a la manzanilla, durazno, peras. En el paladar es fresco, elegante con una pequeña cantidad de azúcar residual que es balanceado por la acidez.
descripcion: ELABORACIÓN DEL VINO   Después de la vendimia manual, la uva es separada de los raspones y son estrujadas suavemente. La fermentación toma lugar en tanques de acero inoxidable, a temperatura controlada para preservar los aromas. La segunda fermentación se lleva a cabo en autoclaves de acero por aproximadamente 35 días. VARIEDAD DE UVA Y ÁREA DE PRODUCCIÓN   Glera 100%. Los viñedos se encuentran en las colinas de Valdobbiadene, la patria del Prosecco, que son las burbujas más famosas de Italia. Aromas inconfundibles de este estilo de vino. La región es p

In [10]:
items

[{'id': '6823fa2dd9f312dbdbb2de73',
  'cmsLocaleId': '664d95581633c04b8a3808b5',
  'lastPublished': '2025-05-14T02:04:29.094Z',
  'lastUpdated': '2025-05-14T02:04:29.094Z',
  'createdOn': '2025-05-14T02:04:29.094Z',
  'isArchived': False,
  'isDraft': False,
  'fieldData': {'descuento': False,
   'descuento-2x1': False,
   'descuento-3x2': False,
   'productoreserva': False,
   'descuento-off': False,
   'precio-descuento': '172000',
   'gr-ml': 'Mililitro por $245,71',
   'precio': '137600',
   'pasillo': 'Vinos',
   'bodega': 'Silvio Carta',
   'name': 'Vermouth Blanco',
   'descripcion': '<p>Esta inspirado en las recetas clásicas de principios del siglo XX. Sunombre y estética evocan la elegancia retro y el espíritu lúdico delos bares italianos de la década de 1920, elaborado coningredientes de origen sardo. Cuenta con Infusión de hierbasaromáticas, cáscaras de cítricos sardos, flores blancas y especiasligeras.Perfecto como aperitivo con hielo y una rodaja de limón, o encócteles com

In [25]:
import numpy as np
import os
import re
import logging
import traceback
from typing import List
import requests

from qdrant_client.http.models import VectorParams, Distance
from qdrant_client import QdrantClient
from openai import OpenAI
from dotenv import load_dotenv
from fastapi import HTTPException


WEBFLOW_API_TOKEN = "026a04fef179155b6a04fbfd49e07c722e7621b91ad98961f6f298987c070180"
COLLECTION_ID = "6660d3a96fe3b376c162563e"
BASE_URL = f"https://api.webflow.com/v2/collections/{COLLECTION_ID}/items"

HEADERS = {
    "Authorization": f"Bearer {WEBFLOW_API_TOKEN}",
    "accept-version": "2.0.0"
}

def get_all_webflow_items():
    offset = 0
    limit = 100
    all_items = []

    while True:
        response = requests.get(
            BASE_URL,
            headers=HEADERS,
            params={"offset": offset, "limit": limit}
        )
        response.raise_for_status()
        data = response.json()
        items = data.get("items", [])

        all_items.extend(items)
        print(f"✅ Fetched {len(items)} items at offset {offset}")

        if len(items) < limit:
            break

        offset += limit

    return all_items

# Define top-level fields (outside fieldData)
top_fields = [
    "id",
    "lastPublished",
    "lastUpdated",
    "isArchived",
    "isDraft"
]

# Define selected fieldData fields
fielddata_fields = [
    "gr-ml",
    "ocasion",  # make sure spelling is consistent with Webflow — was 'ocasion' or 'occasion'?
    "precio",
    "maridaje-1",
    "maridaje-2",
    "pasillo",
    "bodega",
    "name",
    "descripcion",
    "notas-de-cata",
    "tipo",
    "slug",
    "descuento",
    "descuento-2x1",
    "descuento-3x2",
    "productoreserva",
    "descuento-off"
]

# Function to flatten and extract only desired fields
def extract_selected_fields(item):
    row = {}

    # Top-level fields
    for field in top_fields:
        row[field] = item.get(field)

    # From fieldData
    field_data = item.get("fieldData", {})
    for field in fielddata_fields:
        row[field] = field_data.get(field)
    
    # ✅ Extract image URL from 'imagen-del-producto'
    image_info = field_data.get("imagen-del-producto", {})
    if isinstance(image_info, dict):
        row["imagen_url"] = image_info.get("url")
    else:
        row["imagen_url"] = None

    return row


# Get and transform all items
items = get_all_webflow_items()
flattened_data = [extract_selected_fields(item) for item in items]

# Create the DataFrame
df = pd.DataFrame(flattened_data)

# Utility functions
def clean(text):
    if pd.isna(text): return ""
    return str(text).strip()

def strip_html(text):
    return BeautifulSoup(text, "html.parser").get_text(separator=" ", strip=True)


def get_openai_embedding(text: str) -> list:
    response = openai_client.embeddings.create(
        model="text-embedding-3-small",
        input=text
    )
    return response.data[0].embedding

# Connect to Qdrant
client = QdrantClient(host="vps.maestri.com.co", port=6333, https=False)
collection_name = "maestri_products"
embedding_size = 1536  # ✅ OpenAI embedding size is 1536

client.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=embedding_size, distance=Distance.COSINE),
)

points = []

for _, row in df.iterrows():
    if pd.isna(row.get("name")) or pd.isna(row.get("precio")) or row.get("isArchived") is True or row.get("isDraft") is True or row.get("lastPublished") is None:
        continue

    product_name = clean(row.get("name"))
    bodega = clean(row.get("bodega"))
    tipo = clean(row.get("tipo"))
    maridaje1 = clean(row.get("maridaje-1"))
    maridaje2 = clean(row.get("maridaje-2"))
    maridaje = " y ".join([m for m in [maridaje1, maridaje2] if m])
    notas = clean(row.get("notas-de-cata"))
    descripcion = strip_html(clean(row.get("descripcion")))
    precio = clean(row.get("precio"))
    category = clean(row.get("pasillo"))
    gr_ml = clean(row.get("gr-ml"))
    ocasion = clean(row.get("ocasion"))
    slug = clean(row.get("slug"))
    url = f"https://maestri.com.co/products/{slug}" if slug else ""

    descuento = bool(row.get("descuento", False))
    descuento_2x1 = bool(row.get("descuento-2x1", False))
    descuento_3x2 = bool(row.get("descuento-3x2", False))
    productoreserva = bool(row.get("productoreserva", False))
    descuento_off = bool(row.get("descuento-off", False))
    url_imagen = row.get("imagen_url", "")
    if pd.isna(url_imagen):
        url_imagen = ""

    alternate_names = ""

    short_text = f"""Producto: {product_name}. Tipo: {tipo}. Bodega: {bodega}.
    Maridaje: {maridaje}. Notas: {notas}. Descripción: {descripcion}.
    También conocido como: {alternate_names}. Precio: {precio}. GR/ML: {gr_ml}. Ocasion: {ocasion}.
    """

    if not short_text.strip():
        continue

    # Get vector
    vector = get_openai_embedding(short_text)

    # Payload for Qdrant
    payload = {
        "product_name": product_name,
        "bodega": bodega,
        "tipo": tipo,
        "precio": precio,
        "notas": notas,
        "descripcion": descripcion,
        "maridaje": maridaje,
        "category": category,
        "gr_ml": gr_ml,
        "ocasion": ocasion,
        "url": url,
        "descuento": descuento,
        "descuento_2x1": descuento_2x1,
        "descuento_3x2": descuento_3x2,
        "productoreserva": productoreserva,
        "descuento_off": descuento_off,
        "alternate_names": alternate_names,
        "url_imagen": url_imagen
    }


    points.append(PointStruct(id=str(uuid.uuid4()), vector=vector, payload=payload))

# Insert into Qdrant
client.upsert(collection_name=collection_name, points=points)
print(f"✅ Inserted {len(points)} products into Qdrant collection: {collection_name}")


✅ Fetched 100 items at offset 0
✅ Fetched 100 items at offset 100
✅ Fetched 100 items at offset 200
✅ Fetched 100 items at offset 300
✅ Fetched 20 items at offset 400


C:\Users\dhernandez\AppData\Local\Temp\ipykernel_32868\4232199426.py:131: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


✅ Inserted 189 products into Qdrant collection: maestri_products


In [7]:
items

[{'id': '6823fa2dd9f312dbdbb2de73',
  'cmsLocaleId': '664d95581633c04b8a3808b5',
  'lastPublished': '2025-05-14T02:04:29.094Z',
  'lastUpdated': '2025-05-14T02:04:29.094Z',
  'createdOn': '2025-05-14T02:04:29.094Z',
  'isArchived': False,
  'isDraft': False,
  'fieldData': {'descuento': False,
   'descuento-2x1': False,
   'descuento-3x2': False,
   'productoreserva': False,
   'descuento-off': False,
   'precio-descuento': '172000',
   'gr-ml': 'Mililitro por $245,71',
   'precio': '137600',
   'pasillo': 'Vinos',
   'bodega': 'Silvio Carta',
   'name': 'Vermouth Blanco',
   'descripcion': '<p>Esta inspirado en las recetas clásicas de principios del siglo XX. Sunombre y estética evocan la elegancia retro y el espíritu lúdico delos bares italianos de la década de 1920, elaborado coningredientes de origen sardo. Cuenta con Infusión de hierbasaromáticas, cáscaras de cítricos sardos, flores blancas y especiasligeras.Perfecto como aperitivo con hielo y una rodaja de limón, o encócteles com

✅ Fetched 100 items at offset 0
✅ Fetched 100 items at offset 100
✅ Fetched 100 items at offset 200
✅ Fetched 100 items at offset 300
✅ Fetched 20 items at offset 400


In [9]:
df

,id,lastPublished,lastUpdated,isArchived,isDraft,gr-ml,ocasion,precio,maridaje-1,maridaje-2,...,name,descripcion,notas-de-cata,tipo,slug,descuento,descuento-2x1,descuento-3x2,productoreserva,descuento-off
0,6823fa2dd9f312dbdbb2de73,2025-05-14T02:04:29.094Z,2025-05-14T02:04:29.094Z,False,False,"Mililitro por $245,71",None,137600,None,None,...,Vermouth Blanco,<p>Esta inspirado en las recetas clásicas de p...,None,None,vermouth-blanco,False,False,False,False,False
1,6823f83f37e7dd18f6e1d4ed,2025-05-14T12:37:08.834Z,2025-05-14T12:37:08.834Z,False,False,"Mililitro por $245,71",None,137600,None,None,...,Vermouth Rojo,<p>Se elabora a partir de Vernaccia di Oristan...,None,None,vermouth-rojo,False,False,False,False,False
2,6823f71280a8d9de4f1a43f8,2025-05-14T01:58:12.446Z,2025-05-14T01:58:12.446Z,False,False,"Mililitro por $302,85",None,169600,None,None,...,Gin Pigskin,<p>Contiene botánicos locales como enebro sard...,None,None,gin-pigskin,False,False,False,False,False
3,6823f68302f24dd9e4b9df4f,2025-05-14T01:59:01.100Z,2025-05-14T01:59:01.100Z,False,False,"$327,14 por ml",None,183200,None,None,...,Boigin Saffron,"<p>Utiliza azafrán sardo de alta calidad, lo q...",None,None,boigin-saffron,False,False,False,False,False
4,67e03ce522349689dfa5d528,2025-05-05T23:32:45.915Z,2025-04-29T15:40:48.836Z,False,False,None,None,46000,None,None,...,Mascarpone 500gr,"<p>El mascarpone, típicamente lombardo,se elab...",None,None,mascarpone-500gr,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,666150354f742af9a75e85b3,2025-03-12T23:27:56.606Z,2025-03-12T23:27:56.606Z,False,False,Mililitro a $205,"Cena entre amigos, Cena formal, Entre apasiona...",154000,Pasta y Pizza,Sushi,...,"Cannonau ""Sincaru""","<p id="""">ELABORACIÓN DEL VINO &nbsp; Vendimia ...",Color rojo rubí con reflejos violáceos. En la ...,Tinto,sincaru-cannonau-di-sardegna-doc,False,False,False,False,False
416,66615035a7cc81d396c88077,None,2025-02-07T04:30:44.636Z,True,True,Mililitro a $435,"Cena formal, Entre apasionados, Ocasión especi...",163000,Todas las comidas,Carne,...,Barolo 375ml,"<p id="""">ELABORACIÓN DEL VINO &nbsp; La uva es...","De un color rojo granada, con un aroma etéreo,...",Tinto,barolo-2012-docg---media-botella-375ml,False,False,False,False,False
417,66615035437397bdd5e69b3f,2025-03-12T23:21:57.135Z,2025-03-12T23:21:57.135Z,False,False,Mililitro a $143,"Cena entre amigos, Para todos los días, Todas ...",69000,Carne,Mexicano,...,"Cannonau ""Surrau”","<p id="""">ELABORACIÓN DEL VINO &nbsp; Vendimia ...",Color rojo rubí con reflejos granate. En la na...,Tinto,surrau-cannonau-di-sardegna-doc,False,False,False,False,False
418,6661503579739759bbd7e0b5,None,2025-02-22T02:05:45.164Z,True,True,Mililitro a $163,"Cena entre amigos, Cena formal, Entre apasiona...",122000,Todas las comidas,Carne,...,Chianti Poggio Al Vento,"<p id="""">ELABORACIÓN DEL VINO &nbsp; Vendimia ...",Color rojo rubí brillante. En la nariz es perf...,Tinto,poggio-al-vento-chianti-colli-senesi-docg,False,False,False,False,True
